In [1]:
import pprint as pp

In [2]:
#from badassdatascience.forex.database.populate_and_update.mongodb.oop_get_candles import CandlePull

In [94]:
import json
import requests
import datetime
import time
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from badassdatascience.forex.utilities.oanda_tools import get_oanda_headers
from badassdatascience.forex.utilities.oanda_tools import price_type_map

class CandlePull():

    #
    # Constructor
    #
    def __init__(
        self,
        config_file,
        count,
        granularity,
        instrument,
        price_types,
        error_retry_interval = 5,
        keep_complete_only = True,
        verbose = False,
    ):

        #
        # command line arguments
        #
        self.config_file = config_file
        self.count = count
        self.granularity = granularity
        self.instrument = instrument
        self.price_types = price_types
        self.error_retry_interval = error_retry_interval
        self.keep_complete_only = keep_complete_only
        self.verbose = verbose

        #
        # initialize (hard-coded)
        #
        self.timezone_to_use = 'America/Toronto'   # Don't change this!
        self.start_time = int(datetime.datetime(2009, 12, 31, 23, 59, 59, tzinfo=datetime.timezone.utc).timestamp())
        
        #
        # additional initialization
        #
        self.timezone = pytz.timezone(self.timezone_to_use)  # not sure we are using this... check later
        self.price_type_list = [price_type_map[q] for q in self.price_types]
        self.end_date_original = int(time.mktime(datetime.datetime.now().timetuple()))

        #
        # MongoDB
        #
        client = MongoClient()
        self.db = client.forex
    
    #
    # Get the header information needed for Oanda requests
    #
    def get_headers(self):
        with open(self.config_file) as f:
            self.config = json.load(f)
        self.headers = get_oanda_headers(self.config)

    #
    # request forex price/volume candlesticks from Oanda
    #
    def get_instrument_candlesticks(self, end_date):
        url = (
            self.config['server']
            + '/v3/instruments/' + self.instrument
            + '/candles?count=' + str(self.count)
            + '&price=' + self.price_types
            + '&granularity=' + self.granularity
            + '&to=' + str(end_date)
        )
        
        worked = False
        while not worked:
            try:
                r = requests.get(url, headers = self.headers)
                worked = True
            except:
                time.sleep(error_retry_interval)
        
        rj = r.json()
        return rj

    #
    # compute additional forex candlestick features
    #
    def compute_candle_features(self):
        
        finished = False
        end_date = self.end_date_original

        self.insert_many_list = []

        # loop through the timestamp ranges for each set of n=count values
        while not finished:

            # retrieve the instrument candlesticks from the Oanda server
            rj = self.get_instrument_candlesticks(end_date) # instrument, count, price_types, granularity, end_date)        
            candlesticks = rj['candles']

            #
            # deal with timestamps and time-related content
            #
            date_list = []
            for candle in candlesticks:

                candle['instrument'] = self.instrument.replace('_', '/')
                candle['granularity'] = self.granularity
                candle['time'] = int(float(candle['time']))
                time_dt = datetime.datetime.fromtimestamp(candle['time'], tz = self.timezone)
                candle['time_iso'] = time_dt.isoformat()
                candle['weekday'] = time_dt.weekday()
                candle['hour'] = time_dt.hour

                for price_type in self.price_type_list:
                    for candlestick_component in candle[price_type].keys():
                        candle[price_type + '_' + candlestick_component] = float(candle[price_type][candlestick_component])
                    candle[price_type + '_return'] = candle[price_type + '_c'] - candle[price_type + '_o']
                    candle[price_type + '_volatility'] = candle[price_type + '_h'] - candle[price_type + '_l']
            
                for price_type in self.price_type_list:
                    del(candle[price_type])

                
                if self.keep_complete_only:
                    if candle['complete']:    
                        self.insert_many_list.append(candle)
                else:
                    self.insert_many_list.append(candle)

                date_list.append(candle['time'])


            # Are we done?
            if (len(date_list) < self.count) or (min(date_list) < self.start_time):
                finished = True
            else:
                # prepare for the next iteration
                end_date = min(date_list) - 0.1

    #
    # Create a dataframe
    #
    def create_dataframe(self):
        self.df = pd.DataFrame(self.insert_many_list).sort_values(by = ['instrument', 'time'])
        self.df = self.df[self.df['time'] >= int(self.start_time)]
        self.df = self.df.reset_index().drop(columns = ['index']).copy()

    #
    # QA
    #
    # Change this to include assert statements
    #
    def qa(self):
        if self.verbose:
            print(len(self.df.index) == len(self.df[['time']].drop_duplicates()))
            print(len(self.df.index) == len(self.df['time'].unique()))
    
    #
    # See what is already in the database
    #
    def get_max_previous_time(self):
        candlesticks = self.db.candlesticks
        
        result = candlesticks.aggregate(
            [
                {
                    '$match': {'instrument' : self.instrument.replace('_', '/') },
                },
                {
                    '$group': {
                        '_id': None,
                        'max_time' : {'$max': '$time'},
                    },
                },
            ]
        )
        result_list = [q for q in result]
        if len(result_list) > 0:
            self.start_time = result_list[0]['max_time']
    
    #
    # Compute everything
    #
    def fit(self):

        self.get_max_previous_time()
        self.get_headers()
        self.compute_candle_features()
        self.create_dataframe()
        self.qa()

        self.time_filtered_df = self.df[self.df['time'] > self.start_time].sort_values(by = ['time']).copy()
        self.to_insert = self.time_filtered_df.to_dict(orient = 'records')
    

    #
    # plot
    #
    def plot(self, savepath = None):
        plt.figure()
        plt.plot(self.df['time'], self.df['mid_c'])  # we may want to have the plot column be flexible (defaulting to 'mid_c' though)
        plt.title(self.instrument)

        if savepath == None:
            plt.show()
        else:
            plt.savefig(savepath)

        plt.close()

In [89]:
# This cell is tagged `parameters`

config_file = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forex/data/DEVELOPMENT.json'
count = 5000 
granularity = 'M15'

instruments = 'EUR_USD,USD_CAD,USD_JPY,USD_CHF,AUD_USD,GBP_USD,NZD_USD'

price_types = 'BAM'
error_retry_interval = 2

In [90]:
instrument_list = [q.strip() for q in instruments.split(',') if not q.strip() == '']

In [91]:
print(count)
print(granularity)
print(instrument_list)

5000
M15
['EUR_USD', 'USD_CAD', 'USD_JPY', 'USD_CHF', 'AUD_USD', 'GBP_USD', 'NZD_USD']


In [92]:
instrument_dict = {}
for instrument in instrument_list:
    instrument_dict[instrument] = CandlePull(
        config_file, count, granularity, instrument, price_types, error_retry_interval = error_retry_interval,
    )
    instrument_dict[instrument].fit()

    print(instrument + ' is complete.')

    break


[]

EUR_USD is complete.


In [93]:
pp.pprint(instrument_dict['EUR_USD'].to_insert[0:2])

AttributeError: 'CandlePull' object has no attribute 'to_insert'

In [75]:
len(instrument_dict['EUR_USD'].to_insert)

383592

In [76]:
instrument_dict['EUR_USD'].df

,complete,volume,time,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_l,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility
0,True,3,1262540700,EUR/USD,M15,2010-01-03T12:45:00-05:00,6,12,1.43070,1.43097,...,1.43170,1.43247,0.00077,0.00077,1.43120,1.43172,1.43120,1.43172,0.00052,0.00052
1,True,36,1262541600,EUR/USD,M15,2010-01-03T13:00:00-05:00,6,13,1.43097,1.43325,...,1.43185,1.43185,-0.00062,0.00340,1.43172,1.43425,1.43105,1.43110,-0.00062,0.00320
2,True,6,1262542500,EUR/USD,M15,2010-01-03T13:15:00-05:00,6,13,1.43075,1.43080,...,1.43205,1.43230,0.00005,0.00025,1.43150,1.43155,1.43130,1.43155,0.00005,0.00025
3,True,40,1262543400,EUR/USD,M15,2010-01-03T13:30:00-05:00,6,13,1.43100,1.43165,...,1.43181,1.43181,-0.00069,0.00134,1.43175,1.43240,1.43106,1.43106,-0.00069,0.00134
4,True,55,1262544300,EUR/USD,M15,2010-01-03T13:45:00-05:00,6,13,1.43036,1.43097,...,1.43177,1.43222,0.00036,0.00045,1.43111,1.43157,1.43106,1.43157,0.00046,0.00051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383587,True,3623,1745290800,EUR/USD,M15,2025-04-21T23:00:00-04:00,0,23,1.15197,1.15394,...,1.15198,1.15358,0.00143,0.00213,1.15206,1.15402,1.15190,1.15350,0.00144,0.00212
383588,True,2655,1745291700,EUR/USD,M15,2025-04-21T23:15:00-04:00,0,23,1.15341,1.15396,...,1.15348,1.15378,0.00021,0.00065,1.15349,1.15404,1.15340,1.15369,0.00020,0.00064
383589,True,2364,1745292600,EUR/USD,M15,2025-04-21T23:30:00-04:00,0,23,1.15362,1.15401,...,1.15319,1.15402,0.00024,0.00098,1.15370,1.15409,1.15310,1.15394,0.00024,0.00099
383590,True,1625,1745293500,EUR/USD,M15,2025-04-21T23:45:00-04:00,0,23,1.15389,1.15394,...,1.15332,1.15357,-0.00046,0.00079,1.15396,1.15402,1.15324,1.15348,-0.00048,0.00078


In [77]:
instrument_dict['EUR_USD'].time_filtered_df

,complete,volume,time,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_l,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility
0,True,3,1262540700,EUR/USD,M15,2010-01-03T12:45:00-05:00,6,12,1.43070,1.43097,...,1.43170,1.43247,0.00077,0.00077,1.43120,1.43172,1.43120,1.43172,0.00052,0.00052
1,True,36,1262541600,EUR/USD,M15,2010-01-03T13:00:00-05:00,6,13,1.43097,1.43325,...,1.43185,1.43185,-0.00062,0.00340,1.43172,1.43425,1.43105,1.43110,-0.00062,0.00320
2,True,6,1262542500,EUR/USD,M15,2010-01-03T13:15:00-05:00,6,13,1.43075,1.43080,...,1.43205,1.43230,0.00005,0.00025,1.43150,1.43155,1.43130,1.43155,0.00005,0.00025
3,True,40,1262543400,EUR/USD,M15,2010-01-03T13:30:00-05:00,6,13,1.43100,1.43165,...,1.43181,1.43181,-0.00069,0.00134,1.43175,1.43240,1.43106,1.43106,-0.00069,0.00134
4,True,55,1262544300,EUR/USD,M15,2010-01-03T13:45:00-05:00,6,13,1.43036,1.43097,...,1.43177,1.43222,0.00036,0.00045,1.43111,1.43157,1.43106,1.43157,0.00046,0.00051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383587,True,3623,1745290800,EUR/USD,M15,2025-04-21T23:00:00-04:00,0,23,1.15197,1.15394,...,1.15198,1.15358,0.00143,0.00213,1.15206,1.15402,1.15190,1.15350,0.00144,0.00212
383588,True,2655,1745291700,EUR/USD,M15,2025-04-21T23:15:00-04:00,0,23,1.15341,1.15396,...,1.15348,1.15378,0.00021,0.00065,1.15349,1.15404,1.15340,1.15369,0.00020,0.00064
383589,True,2364,1745292600,EUR/USD,M15,2025-04-21T23:30:00-04:00,0,23,1.15362,1.15401,...,1.15319,1.15402,0.00024,0.00098,1.15370,1.15409,1.15310,1.15394,0.00024,0.00099
383590,True,1625,1745293500,EUR/USD,M15,2025-04-21T23:45:00-04:00,0,23,1.15389,1.15394,...,1.15332,1.15357,-0.00046,0.00079,1.15396,1.15402,1.15324,1.15348,-0.00048,0.00078


In [9]:
#for instrument in instrument_list:
#    #print(instrument_dict[instrument].df.head())
#    instrument_dict[instrument].plot()

In [10]:
#import pickle
#with open('output/temp_M15.pickled', 'wb') as fff:
#    pickle.dump(instrument_dict, fff)

In [11]:
import pickle
with open('output/temp_M15.pickled', 'rb') as fff:
    instrument_dict = pickle.load(fff)

In [15]:
#import pymongo
from pymongo import MongoClient

client = MongoClient()
db = client.forex
candlesticks = db.candlesticks

In [16]:
candlesticks.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [17]:
for instrument in instrument_list:
    result = candlesticks.insert_many(instrument_dict[instrument].insert_many_list)
    
#print(result)

In [36]:
instrument = 'EUR/USD'
#stuff = db.candlesticks.find( { 'instrument' : instrument }, { 'time' : 1 } )

#times_dict = {}
#for t in stuff:
#    times_dict[t['time']] = None

#the_times = sorted([q['time'] for q in stuff])

result = candlesticks.aggregate([
    {
        '$match': {'instrument' : instrument}  # Filter documents
    },
    {
        '$group': {
            '_id': None,
            'max_time' : {'$max': '$time'},
            'min_time' : {'$min' : '$time'},
        }
    }
])

#for i in result: 
#    print(i)

[q for q in result][0]['max_time']

1745291700